<a href="https://colab.research.google.com/github/sarathb818/Cotton-Leaf-Disease-prediction-using-the-transfer-learning-techniques/blob/main/Transfer_Learning_InceptionV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# To prevent the gpu from overclocking
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5 # This allows only 50 % usage of the GPU.
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
#importing the libraries

from tensorflow.keras.layers import Input,Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential

import numpy as np
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Resizing the all images to default size 
IMAGE_SIZE = [224,224]
train_path = '/content/drive/My Drive/data/train'
valid_path = '/content/drive/My Drive/Test'

In [ ]:
#import the InceptionV3 library and the add preprocessing layer to the front Resnet
# Here we are using the pretrained weights from the imagenet bcz it has huge of catgories with image
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# Since we want to remove the top layer as our image_size we will make the include_top = False

87916544/87910968 [==============================] - 3s 0us/step


In [ ]:
# Since we dont want to train the existing weights bcz we are taking it from the weights of InceptionV3
for layer in inception.layers:
    layer.trainable = False

In [ ]:
#for getting number of output classes
folders = glob('/content/drive/My Drive/data/train/*')

In [ ]:
# Flattening because we add the last number of layers based on the categories of the image
x = Flatten()(inception.output)

In [ ]:
prediction = Dense(len(folders),activation = 'softmax')(x) # This is get the output nodes
# Softmax is used since we have n number of categories

# Create a model 

model = Model(inputs=inception.input,outputs = prediction)

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# The ImageDataGenerator is to import the images from the dataset
# The Dataaugmentation for the train data set

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1/255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/data/train',
                                                 target_size = (224,224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 1951 images belonging to 4 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Test',
                                        target_size = (224,224),
                                        batch_size = 16,
                                        class_mode = 'categorical')


Found 18 images belonging to 4 classes.


In [ ]:
#fitting the model

r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs =20,
    steps_per_epoch =len(training_set),
    validation_steps =len(test_set)
)


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
122/122 [==============================] - 1595s 13s/step - loss: 2.0200 - accuracy: 0.7678 - val_loss: 0.1274 - val_accuracy: 0.9444
Epoch 2/20
122/122 [==============================] - 45s 372ms/step - loss: 1.2158 - accuracy: 0.8662 - val_loss: 0.5025 - val_accuracy: 0.7778
Epoch 3/20
122/122 [==============================] - 45s 371ms/step - loss: 1.2238 - accuracy: 0.8821 - val_loss: 0.9773 - val_accuracy: 0.8889
Epoch 4/20
122/122 [==============================] - 45s 372ms/step - loss: 0.9355 - accuracy: 0.9124 - val_loss: 0.0651 - val_accuracy: 0.9444
Epoch 5/20
122/122 [==============================] - 46s 374ms/step - loss: 0.7501 - accuracy: 0.9257 - val_loss: 0.0055 - val_accuracy: 1.0000
Epoch 6/20
122/122 [==============================] - 45s 371ms/step - loss: 0.7801 - accuracy: 0.9252 - val_loss: 0.8375 - val_accuracy: 0.7778
Epoch 7/20
122/122 [==============================] - 